## Perceptron

Presenta: Carlos Rábago
    
El presente algoritmo muestra la aplicacion del Perceptron, describi paso a paso cada parte del codigo, como una forma 
de aprendizaje, anexo los datos del autor.    

Autor del algoritmo Dilan Coss

Rescatado de: https://blog.nearsoftjobs.com/c%C3%B3mo-hacer-un-perceptr%C3%B3n-907a1020e5af        

Se trabajara con un set de datos conocido llamado IRIS , cuenta con 4 atributos (x1,x2,x3 y x4) y 150 datos a utilizar.
Este set de datos tiene 3 especies de plantas, pero nuestro perceptrón es un clasificador binario, se escogera dos tipos de especies iris-setosa e iris-versicolor.

El algoritmo se dividira en los siguientes pasos.

1. Necesitamos dividir nuestras muestras en un set de entrenamiento y otro de verificación. En este caso 90% de entrenamiento y 10% de verificación. en este caso por ser pocos datos.

2. Asignar cada especie de planta a una clase, con -1 y 1.

3. Generar weights aleatorios

4. Tratar de predecir con nuestra ecuación w0*bias + w1*x1 + w2*x2.

5. Si lo predice mal, debemos ajustar los weights con:
    w0 = w0 + clase*bias
    wn = wn + clase*xn

Nota: nunca verifiques la presicion del modelo con los datos con los que entrenaste, puesto que el sentido es validar (probar que puede identificar con datos similares)


Se cargan librerias

In [15]:
import csv
import random

#### Perceptron: 

Es un clasificador binario, es decir que es capaz de discriminar solamente entre dos categorias y estas categorias deben ser linealmente separables, es decir que si los datos son graficados en un plano, pueden ser divididos por una recta.


Se definiran los siguientes metodos para la clase perceptron

* generate_random_weights
* train
* predict
* adjust_weights
* verify

In [16]:
# El constructor de la clase Perceptrón recibirá el número de weights y un array de las dos clases que se mapean a -1 y 1.

class Perceptron:
    def __init__(self, number_of_weights, classes):
        self.number_of_weights = number_of_weights
        self.weights = self.generate_random_weights(number_of_weights)
        self.dict_classes = { classes[0]:1, classes[1]:-1 }

    # La generación de weights aleatorios es muy sencilla, sólo intenta que abarquen números negativos y positivos, 
    # el rango de -5 a 5
    
    def generate_random_weights(self, n):
        weights = []
        for x in range(n):
            weights.append(random.random()*10-5)
        print('Initial weights: {}'.format(weights))
        return weights

    # el entrenamiento consiste en hacer una predicción, si ésta es correcta seguimos con lo siguientes datos, 
    # si es incorrecta ajustamos los weights
    # cuenta con un atributo epochs, el cual indica cuántas veces iterará la misma información para ajustar cada vez 
    # más los weights.
    
    def train(self, data, epochs):
        for epoch in range(epochs):
            print('Epoch {}'.format(epoch))
            for datum in data:
                real_class = self.dict_classes[datum[len(datum)-1]]
                prediction_class = self.predict(datum)
                #print('y:{}, prediction_class:{}'.format(y,prediction_class))
                if real_class != prediction_class:
                    self.adjust_weights(real_class,datum)
                    print('Adjusted weights: {}'.format(self.weights))
            print('Final weights from epoch {}: {}'.format(epoch,self.weights))

    # La predicción es la suma de las multiplicaciones de los atributos por los weights más el bias 
    # (el bias lo deje como el primer elemento del array weights). Si el resultado es positivo predecimos con un 1, 
    # de lo contrario predecimos con un -1.
    
    def predict(self, datum):
        weights_without_bias = self.weights[1:self.number_of_weights]
        attribute_values = datum[:self.number_of_weights-1]
        weight_bias = self.weights[0]
        activation = sum([i*j for i,j in zip(weights_without_bias,attribute_values)]) + weight_bias
        return 1 if activation > 0 else -1
  
    # ¿Qué pasa si nuestra predicción es incorrecta?, tenemos que ajustar los weights
    # w0 = w0 + clase*bias
    # wn = wn + clase*xn
    # La w es el weight, la clase es -1 o 1, x es el atributo y el bias siempre es 1.
    
    #Puedes notar que el único diferente es el bias, que lo que hacemos es simplemente sumarle la clase real. 
    # Los demás weights se calculan al sumarle la multiplicación de la clase real con el valor del atributo.
    
    def adjust_weights(self, real_class, datum):
        self.weights[0] = self.weights[0] + real_class
        for i in range(1,self.number_of_weights):
            self.weights[i] = self.weights[i] + real_class * datum[i-1]

    # el método de verificación, qué es exactamente lo mismo que en el entrenamiento, sólo que aquí ya no se 
    # ajustan los weights. El método regresará en porcentaje cuántas predicciones fueron correctas.
    
    def verify(self, data):
        count = 0
        for datum in data:
            real_class = self.dict_classes[datum[len(datum)-1]]
            prediction_class = self.predict(datum)
            if real_class != prediction_class:
                count = count + 1
        return (1-count/len(data))*100

In [17]:
# Se define la funcion para la lectura de los datos

def read_csv(file):
    with open(file,'r') as f:
        reader = csv.reader(f)
        info = []
        for row in reader:
             info.append(row)

        for datum in info:
            for i in range(len(info[0])-1):
                datum[i] = float(datum[i])
        return info

El siguiente método nos regresará un array con 150 elementos, 50 de cada especie, los podemos separar para trabajar con ellos en las combinaciones que queramos, por ejemplo utilizaré las especies iris-setosa e iris-versicolor para iniciar. Haré un ordenamiento aleatorio porque en realidad no debe importar el orden en el que vengan los datos, por último separaré en 90% y 10% para el entrenamiento y verificación respectivamente.

In [18]:
def main():
    # leyendo los datos del archivo 
    data = read_csv('iris_dataset.csv')
    data_iris_setosa = data[0:50]              # los primeros 50 datos corresponden a la especie Iris-setosa
    data_iris_versicolor = data[50:100]        # los datos de 51 al 100 corresponden a la especie Iris-versicolor
    data_iris_virginica = data[100:150]        # los datos de 101 al 150 corresponden a la especie Iris-virginica

    # utilizaremos las especies Iris-virginica e iris-versicolor para iniciar.
    data_with_two_species = data_iris_versicolor + data_iris_virginica
    classes = ['Iris-versicolor', 'Iris-virginica']

    # Ordenar los datos de forma aleatoria
    random.shuffle(data_with_two_species)

    # Entrenamiento y verificacion de datos
    data_training = data_with_two_species[0:90]
    data_verification = data_with_two_species[:-10]

    # Perceptron
    perceptron = Perceptron(len(data[0]), classes)

    # Entrenamiento
    perceptron.train(data_training, epochs=1000)
    print('\nFinal weights from training: {}'.format(perceptron.weights))

    # Verificacion
    accuracy = perceptron.verify(data_verification)
    print('Error: {} %'.format(100-accuracy))


In [19]:
if __name__ == "__main__":
    main()

Initial weights: [-0.5234185253238852, -0.2504701214821514, 4.00314295243594, 2.0230571821078582, 2.965269482757087]
Epoch 0
Adjusted weights: [-1.5234185253238852, -6.250470121482151, 1.8031429524359401, -2.9769428178921418, 1.465269482757087]
Adjusted weights: [-0.5234185253238852, -0.6504701214821518, 4.50314295243594, 1.2230571821078584, 2.7652694827570867]
Adjusted weights: [-1.5234185253238852, -6.250470121482151, 1.7031429524359405, -3.676942817892142, 0.7652694827570867]
Adjusted weights: [-0.5234185253238852, -0.5504701214821512, 4.50314295243594, 0.4230571821078577, 2.0652694827570865]
Adjusted weights: [-1.5234185253238852, -6.850470121482151, 1.2031429524359405, -5.576942817892142, -0.43473051724291345]
Adjusted weights: [-0.5234185253238852, -0.8504701214821511, 3.9031429524359407, -0.47694281789214266, 1.1652694827570866]
Adjusted weights: [-1.5234185253238852, -7.750470121482151, 0.8031429524359406, -5.876942817892143, -0.9347305172429135]
Adjusted weights: [-0.523418525

Adjusted weights: [42.47658147467612, 50.94952987851793, 41.303142952436, -51.87694281789212, -112.13473051724282]
Adjusted weights: [41.47658147467612, 44.64952987851793, 38.603142952435995, -56.77694281789212, -113.93473051724281]
Final weights from epoch 58: [41.47658147467612, 44.64952987851793, 38.603142952435995, -56.77694281789212, -113.93473051724281]
Epoch 59
Adjusted weights: [42.47658147467612, 51.349529878517934, 41.603142952435995, -51.77694281789212, -112.23473051724281]
Adjusted weights: [41.47658147467612, 45.349529878517934, 39.40314295243599, -56.77694281789212, -113.73473051724281]
Adjusted weights: [42.47658147467612, 51.349529878517934, 42.103142952435995, -51.67694281789212, -112.13473051724282]
Adjusted weights: [41.47658147467612, 44.449529878517936, 39.003142952435994, -57.07694281789212, -114.23473051724281]
Adjusted weights: [42.47658147467612, 50.64952987851794, 41.203142952436, -52.57694281789212, -112.73473051724281]
Adjusted weights: [41.47658147467612, 4

Adjusted weights: [53.47658147467612, 53.749529878518, 55.60314295243621, -55.176942817891984, -144.73473051724346]
Adjusted weights: [52.47658147467612, 47.649529878517995, 52.60314295243621, -60.07694281789198, -146.53473051724347]
Adjusted weights: [53.47658147467612, 53.94952987851799, 55.10314295243621, -55.176942817891984, -145.03473051724347]
Adjusted weights: [52.47658147467612, 47.74952987851799, 52.30314295243621, -59.97694281789198, -146.8347305172435]
Adjusted weights: [53.47658147467612, 53.64952987851799, 55.503142952436214, -55.176942817891984, -145.03473051724347]
Adjusted weights: [52.47658147467612, 47.64952987851799, 52.503142952436214, -59.97694281789198, -146.8347305172435]
Adjusted weights: [53.47658147467612, 53.24952987851799, 55.503142952436214, -55.47694281789198, -145.3347305172435]
Adjusted weights: [52.47658147467612, 46.94952987851799, 52.80314295243621, -60.37694281789198, -147.1347305172435]
Final weights from epoch 99: [52.47658147467612, 46.94952987851

Adjusted weights: [57.47658147467612, 50.349529878517934, 65.00314295243636, -62.97694281789178, -171.63473051724503]
Final weights from epoch 138: [57.47658147467612, 50.349529878517934, 65.00314295243636, -62.97694281789178, -171.63473051724503]
Epoch 139
Adjusted weights: [58.47658147467612, 57.04952987851794, 68.00314295243636, -57.97694281789178, -169.93473051724504]
Adjusted weights: [57.47658147467612, 51.04952987851794, 65.80314295243636, -62.97694281789178, -171.43473051724504]
Adjusted weights: [58.47658147467612, 57.04952987851794, 68.50314295243636, -57.87694281789178, -169.83473051724505]
Adjusted weights: [57.47658147467612, 50.949529878517936, 65.50314295243636, -62.77694281789178, -171.63473051724506]
Adjusted weights: [58.47658147467612, 57.24952987851793, 68.00314295243636, -57.87694281789178, -170.13473051724506]
Adjusted weights: [57.47658147467612, 51.04952987851793, 65.20314295243637, -62.67694281789178, -171.93473051724507]
Adjusted weights: [58.47658147467612, 5

Adjusted weights: [60.47658147467612, 58.24952987851778, 71.00314295243685, -65.67694281789183, -198.23473051724687]
Adjusted weights: [61.47658147467612, 64.14952987851778, 74.20314295243685, -60.87694281789183, -196.43473051724686]
Adjusted weights: [60.47658147467612, 58.14952987851778, 71.20314295243685, -65.67694281789183, -198.23473051724687]
Final weights from epoch 189: [60.47658147467612, 58.14952987851778, 71.20314295243685, -65.67694281789183, -198.23473051724687]
Epoch 190
Adjusted weights: [61.47658147467612, 64.84952987851779, 74.20314295243685, -60.67694281789183, -196.53473051724689]
Adjusted weights: [60.47658147467612, 58.849529878517785, 72.00314295243685, -65.67694281789183, -198.03473051724689]
Adjusted weights: [61.47658147467612, 64.84952987851779, 74.70314295243685, -60.576942817891826, -196.4347305172469]
Adjusted weights: [60.47658147467612, 57.94952987851779, 71.60314295243685, -65.97694281789182, -198.53473051724689]
Adjusted weights: [61.47658147467612, 64.

...

Final weights from epoch 695: [78.47658147467612, 82.04952987852093, 89.60314295244102, -71.07694281789053, -302.7347305172513]
Epoch 696
Adjusted weights: [79.47658147467612, 88.04952987852093, 92.30314295244102, -65.97694281789053, -301.1347305172513]
Adjusted weights: [78.47658147467612, 81.94952987852093, 89.30314295244102, -70.87694281789054, -302.9347305172513]
Adjusted weights: [77.47658147467612, 75.64952987852094, 86.50314295244102, -75.97694281789053, -304.4347305172513]
Adjusted weights: [78.47658147467612, 81.94952987852093, 89.00314295244102, -71.07694281789053, -302.9347305172513]
Adjusted weights: [79.47658147467612, 87.84952987852094, 92.20314295244103, -66.27694281789053, -301.1347305172513]
Adjusted weights: [78.47658147467612, 81.84952987852094, 89.20314295244103, -71.07694281789053, -302.9347305172513]
Final weights from epoch 696: [78.47658147467612, 81.84952987852094, 89.20314295244103, -71.07694281789053, -302.9347305172513]
Epoch 697
Adjusted weights: [79.476581

Adjusted weights: [83.47658147467612, 79.34952987852144, 84.30314295244145, -68.87694281789022, -307.9347305172507]
Adjusted weights: [84.47658147467612, 85.24952987852144, 87.50314295244145, -64.07694281789023, -306.1347305172507]
Adjusted weights: [83.47658147467612, 77.94952987852145, 84.60314295244144, -70.37694281789022, -307.9347305172507]
Adjusted weights: [84.47658147467612, 83.54952987852144, 87.60314295244144, -65.87694281789022, -306.4347305172507]
Final weights from epoch 746: [84.47658147467612, 83.54952987852144, 87.60314295244144, -65.87694281789022, -306.4347305172507]
Epoch 747
Adjusted weights: [83.47658147467612, 77.44952987852145, 85.00314295244145, -71.47694281789022, -307.8347305172507]
Adjusted weights: [84.47658147467612, 83.44952987852145, 87.70314295244145, -66.37694281789022, -306.2347305172507]
Adjusted weights: [83.47658147467612, 77.14952987852145, 84.90314295244146, -71.47694281789022, -307.7347305172507]
Adjusted weights: [84.47658147467612, 83.449529878

Adjusted weights: [89.47658147467612, 74.64952987852202, 80.50314295244195, -72.0769428178894, -311.8347305172499]
Adjusted weights: [90.47658147467612, 80.94952987852201, 83.00314295244195, -67.1769428178894, -310.3347305172499]
Adjusted weights: [91.47658147467612, 86.84952987852202, 86.20314295244195, -62.3769428178894, -308.5347305172499]
Adjusted weights: [90.47658147467612, 80.84952987852202, 83.20314295244195, -67.1769428178894, -310.3347305172499]
Final weights from epoch 805: [90.47658147467612, 80.84952987852202, 83.20314295244195, -67.1769428178894, -310.3347305172499]
Epoch 806
Adjusted weights: [91.47658147467612, 86.84952987852202, 85.90314295244195, -62.076942817889396, -308.7347305172499]
Adjusted weights: [90.47658147467612, 80.74952987852203, 82.90314295244195, -66.9769428178894, -310.5347305172499]
Adjusted weights: [89.47658147467612, 74.44952987852203, 80.10314295244196, -72.07694281788939, -312.0347305172499]
Adjusted weights: [90.47658147467612, 80.74952987852203

Adjusted weights: [94.47658147467612, 80.84952987852242, 81.50314295244229, -67.27694281788885, -312.03473051724933]
Final weights from epoch 844: [94.47658147467612, 80.84952987852242, 81.50314295244229, -67.27694281788885, -312.03473051724933]
Epoch 845
Adjusted weights: [95.47658147467612, 86.84952987852242, 84.20314295244229, -62.17694281788885, -310.4347305172493]
Adjusted weights: [94.47658147467612, 80.74952987852242, 81.20314295244229, -67.07694281788885, -312.2347305172493]
Adjusted weights: [93.47658147467612, 74.44952987852243, 78.4031429524423, -72.17694281788884, -313.7347305172493]
Adjusted weights: [94.47658147467612, 80.74952987852242, 80.9031429524423, -67.27694281788884, -312.2347305172493]
Adjusted weights: [95.47658147467612, 86.64952987852243, 84.1031429524423, -62.47694281788884, -310.4347305172493]
Adjusted weights: [94.47658147467612, 80.64952987852243, 81.1031429524423, -67.27694281788884, -312.2347305172493]
Final weights from epoch 845: [94.47658147467612, 80

Adjusted weights: [104.47658147467612, 88.24952987852325, 84.203142952443, -63.576942817887726, -312.13473051724804]
Adjusted weights: [103.47658147467612, 82.24952987852325, 81.203142952443, -68.37694281788772, -313.93473051724806]
Final weights from epoch 925: [103.47658147467612, 82.24952987852325, 81.203142952443, -68.37694281788772, -313.93473051724806]
Epoch 926
Adjusted weights: [104.47658147467612, 88.24952987852325, 83.903142952443, -63.27694281788772, -312.33473051724803]
Adjusted weights: [103.47658147467612, 82.14952987852325, 80.903142952443, -68.17694281788772, -314.13473051724804]
Adjusted weights: [102.47658147467612, 75.84952987852326, 78.10314295244301, -73.27694281788771, -315.63473051724804]
Adjusted weights: [103.47658147467612, 82.14952987852325, 80.60314295244301, -68.37694281788771, -314.13473051724804]
Adjusted weights: [104.47658147467612, 88.04952987852326, 83.80314295244301, -63.57694281788771, -312.33473051724803]
Adjusted weights: [103.47658147467612, 82.0

Se muestra un error de 3.3333333333333286 % ún cuando no son completamente separables por el perceptrón